# API Client
The following tutorial demonstrates how to access the PKDB API with python.

For more information and installation information see:  
https://github.com/matthiaskoenig/pkdb

In [544]:
URL_PKDB = "http://0.0.0.0:8000/"

In [545]:
# import libraries
import coreapi
import requests
import json
from pprint import pprint
from requests.auth import HTTPBasicAuth

In [546]:
#response = requests.get('http://0.0.0.0:8000/', auth=HTTPBasicAuth('janek', 'test'))

In [547]:
#r = requests.post("http://0.0.0.0:8000/api-auth/login/?next=/api/v1/", data={'username': 'janek', 'password': 'test'})
#print(r.status_code, r.reason)
#r.json()["token"]

## Create client & API endpoint

In [548]:
#auth = coreapi.auth.BasicAuthentication(
#username="janek",
#password="test", )
client = coreapi.Client()

In [549]:
# get the api scema
document = client.get(URL_PKDB)
print(document.title)
print(document.url)

PkBD API
http://0.0.0.0:8000/


In [550]:
# overview available methods
print(document)

<PkBD API "http://0.0.0.0:8000/">
    authors: {
        list([page], [first_name], [last_name], [search])
        create([first_name], [last_name])
        read(id, [first_name], [last_name], [search])
        update(id, [first_name], [last_name], [first_name], [last_name], [search])
        partial_update(id, [first_name], [last_name], [first_name], [last_name], [search])
        delete(id, [first_name], [last_name], [search])
    }
    characteristic_values: {
        list([page], [choice], [count], [category], [search])
        create(category, [count], [choice], [value], [mean], [median], [min], [max], [sd], [se], [cv], [unit], [cvtype], [group])
        read(id, [choice], [count], [category], [search])
        update(id, category, [count], [choice], [value], [mean], [median], [min], [max], [sd], [se], [cv], [unit], [cvtype], [group], [choice], [count], [category], [search])
        partial_update(id, [count], [choice], [value], [mean], [median], [min], [max], [sd], [se], [cv], [u

## Example queries
Show all authors

In [551]:
data = client.action(document,["authors", "list"])
print(json.dumps(data, indent=2))

{
  "count": 108,
  "next": "http://0.0.0.0:8000/api/v1/authors/?page=2",
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    },
    {
      "id": 2,
      "first_name": "Christine asdin",
      "last_name": "Haller"
    },
    {
      "id": 43,
      "first_name": "W",
      "last_name": "Zilly"
    },
    {
      "id": 78,
      "first_name": "A H",
      "last_name": "Neims"
    },
    {
      "id": 79,
      "first_name": "M",
      "last_name": "Bonati"
    },
    {
      "id": 44,
      "first_name": "E",
      "last_name": "Richter"
    },
    {
      "id": 80,
      "first_name": "R",
      "last_name": "Latini"
    },
    {
      "id": 81,
      "first_name": "F",
      "last_name": "Galletti"
    },
    {
      "id": 45,
      "first_name": "K-Y",
      "last_name": "Seng"
    },
    {
      "id": 46,
      "first_name": "C-Y",
      "last_name": "Fun"
    }
  ]
}


Create author

In [552]:
data = client.action(document,["authors", "create"], params={'first_name':'Lucek', 'last_name':'Grzegorzewski'})
print(data)

OrderedDict([('id', 1), ('first_name', 'Lucek'), ('last_name', 'Grzegorzewski')])


In [509]:
# Filter for authors first name = Lucek
data = client.action(document, ["authors", "list"], params={"first_name": "Lucek"} )
print(json.dumps(data, indent=2))

{
  "count": 1,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    }
  ]
}


In [553]:
# Search for author
data = client.action(document, ["authors", "list"], params={"search": "Gr"} )
print(json.dumps(data, indent=2))
import numpy as np

{
  "count": 5,
  "next": null,
  "previous": null,
  "results": [
    {
      "id": 6,
      "first_name": "L",
      "last_name": "Granit"
    },
    {
      "id": 55,
      "first_name": "D J",
      "last_name": "Greenblatt"
    },
    {
      "id": 11,
      "first_name": "Marika T",
      "last_name": "Granfors"
    },
    {
      "id": 1,
      "first_name": "Lucek",
      "last_name": "Grzegorzewski"
    },
    {
      "id": 38,
      "first_name": "David J",
      "last_name": "Greenblatt"
    }
  ]
}


In [554]:
# Create study
reference_dict = {
    "pmid": 12087394589,
    "sid": "012087345981",
    "name":"Haller2002",
    "date": "2002-07-18",
    "journal": "Clinical pharmacology and therapeutics",
    "title": "Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.",
    "abstract": "Botanical stimgle dose.",
    "authors": [
        {
            "first_name": "Christine asdin",
            "last_name": "Haller"
        },
        {
            "first_name": "Peyton",
            "last_name": "Jacob"
        },
        {
            "first_name": "Neal L",
            "last_name": "Benowitz"
        }
    ],
    "doi": "",
        
        }
    


client.action(document, ["references", "create"], params=reference_dict)

OrderedDict([('pmid', '12087394589'),
             ('sid', '012087345981'),
             ('name', 'Haller2002'),
             ('doi', ''),
             ('title',
              'Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.'),
             ('abstract', 'Botanical stimgle dose.'),
             ('journal', 'Clinical pharmacology and therapeutics'),
             ('date', '2002-07-18'),
             ('authors',
              [OrderedDict([('id', 2),
                            ('first_name', 'Christine asdin'),
                            ('last_name', 'Haller')]),
               OrderedDict([('id', 3),
                            ('first_name', 'Peyton'),
                            ('last_name', 'Jacob')]),
               OrderedDict([('id', 4),
                            ('first_name', 'Neal L'),
                            ('last_name', 'Benowitz')])])])

In [555]:
client.action(document, ["studies", "list"])

OrderedDict([('count', 26),
             ('next', 'http://0.0.0.0:8000/api/v1/studies/?page=2'),
             ('previous', None),
             ('results',
              [OrderedDict([('sid', '19023902'),
                            ('name', 'test'),
                            ('groups',
                             [OrderedDict([('name', 'Seng2009-S_NCNS'),
                                           ('count', 14),
                                           ('description',
                                            '59 healthy adult Asian volunteers were included. In what follows, a non-caffeine consumer (NCCS) refers to an individual who consumed beverages\n(coffee or tea) containing <=200 mg of caffeine per day, and a regular caffeine consumer to one who consumed more >200 mg. A cup (150 mL) of coffee or tea is equivalent to approximately a caffeine dose of 70 and 25 mg respectively (9). Additionally, a non-smoker refers to an individual who did not smoke for at least 6 months prior

In [556]:
# Create study
study_dict = {
    "sid":"19023902",
    "name":'test',
    "reference":"012087345981",
    "groups": [
        {
            "count": 14,
            "name": "Seng2009-S_NCNS",
            "description": "59 healthy adult Asian volunteers were included. In what follows, a non-caffeine consumer (NCCS) refers to an individual who consumed beverages\n(coffee or tea) containing <=200 mg of caffeine per day, and a regular caffeine consumer to one who consumed more >200 mg. A cup (150 mL) of coffee or tea is equivalent to approximately a caffeine dose of 70 and 25 mg respectively (9). Additionally, a non-smoker refers to an individual who did not smoke for at least 6 months prior to dosing. The study population consisted of 14 non-caffeine consumers, non-smokers (NCNS) [age (mean \u00b1 SD) 21 \u00b1 2 years, body weight 62 \u00b1 9 kg), 15 caffeine consumers, non smokers (CNS) (age 24 \u00b1 4 years, body weight 69 \u00b1 12 kg), and 30 caffeine consumer, smokers (CS) (actual consumption 10 \u00b1 4 cigarettes daily, age 23 \u00b1 5 years, body weight 71 \u00b1 19 kg).",
            "characteristic_values": [
                {
                    "count": 14,
                    "category": "sex",
                    "choice": "M"
                },
                {
                    "count": 14,
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "count": 14,
                    "mean": 62.0,
                    "sd": 9.0,
                    "unit": "kg"
                },
                {
                    "count": 14,
                    "category": "ethnicity",
                    "choice": "Asian"
                },
        
            ]
        },
        
    ],
        
        }
    


In [557]:
study_dict = {
    "sid": 16198659,
    "name": "Granfors2005",
    "reference": 16198659,
    "groups": [
        {
            "count": 15,
            "name": "Granfors2005-S1",
            "description": "This was an open, parallel-group study with 15 healthy female volunteers (mean age, 22+-2 years [range, 18-25 years]; mean weight, 57+-6 kg [range, 48-63 kg]) using Ocs and 15 healthy female volunteers (mean age, 22+-2 years [range, 19-26 years]; mean weight, 62+-10 kg [range, 52-74 kg]) without any concomitant medication.\nThere were no statistically significant differences in age or weight between the groups. The combined OCs used by the women in this study contained 75 g gestodene and either 20 g ethinyl estradiol (Harmonet tablet [Wyeth, Newbridge, Ireland], 7 subjects; Meliane tablet [Schering, Berlin, Germany], 5 subjects) or 30 g ethinyl estradiol (Femoden tablet [Schering], 2 subjects; Minulet tablet [Wyeth], 1 subject). The OC users had been using these OC preparations for at least 1 menstrual cycle before the study. Before entering the study, all subjects provided written informed consent and were ascertained to be healthy by medical history, physical examination, and routine laboratory tests. For safety reasons, subjects with a systolic blood pressure lower than 110 mm Hg were excluded from the study. None of the subjects were tobacco smokers, and none used any continuous medication except OCs.",
            "characteristic_values": [
                {
                    "category": "sex",
                    "choice": "F"
                },
                {
                    "category": "oral_contraceptives",
                    "choice": "N"
                },
                {
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "mean": 62.0,
                    "min": 52.0,
                    "max": 74.0,
                    "sd": 10.0,
                    "unit": "kg"
                },
                {
                    "category": "smoking",
                    "choice": "N"
                },
                {
                    "category": "healthy",
                    "choice": "Y"
                },
                {
                    "category": "age",
                    "mean": 22.0,
                    "min": 19.0,
                    "max": 26.0,
                    "sd": 2.0,
                    "unit": "yr"
                }
            ]
        },
        {
            "count": 15,
            "name": "Granfors2005-S2",
            "description": "This was an open, parallel-group study with 15 healthy female volunteers (mean age, 22+-2 years [range, 18-25 years]; mean weight, 57+-6 kg [range, 48-63 kg]) using Ocs and 15 healthy female volunteers (mean age, 22+-2 years [range, 19-26 years]; mean weight, 62+-10 kg [range, 52-74 kg]) without any concomitant medication.\nThere were no statistically significant differences in age or weight between the groups. The combined OCs used by the women in this study contained 75 g gestodene and either 20 g ethinyl estradiol (Harmonet tablet [Wyeth, Newbridge, Ireland], 7 subjects; Meliane tablet [Schering, Berlin, Germany], 5 subjects) or 30 g ethinyl estradiol (Femoden tablet [Schering], 2 subjects; Minulet tablet [Wyeth], 1 subject). The OC users had been using these OC preparations for at least 1 menstrual cycle before the study. Before entering the study, all subjects provided written informed consent and were ascertained to be healthy by medical history, physical examination, and routine laboratory tests. For safety reasons, subjects with a systolic blood pressure lower than 110 mm Hg were excluded from the study. None of the subjects were tobacco smokers, and none used any continuous medication except OCs.",
            "characteristic_values": [
                {
                    "category": "sex",
                    "choice": "F"
                },
                {
                    "category": "oral_contraceptives",
                    "choice": "Y"
                },
                {
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "mean": 57.0,
                    "min": 48.0,
                    "max": 63.0,
                    "sd": 6.0,
                    "unit": "kg"
                },
                {
                    "category": "smoking",
                    "choice": "N"
                },
                {
                    "category": "healthy",
                    "choice": "Y"
                },
                {
                    "category": "age",
                    "mean": 22.0,
                    "min": 18.0,
                    "max": 25.0,
                    "sd": 2.0,
                    "unit": "yr"
                }
            ]
        }
    ]
}

In [558]:
client.action(document, ["studies", "create"], params=study_dict)

ErrorMessage: <Error: 400 Bad Request>
    groups: [
        {
            characteristic_values: [
                {},
                {},
                {},
                {
                    category: [
                        "\"body_weight\" is not a valid choice."
                    ]
                },
                {},
                {},
                {}
            ]
            intervention: [
                "This field is required."
            ]
        },
        {
            characteristic_values: [
                {},
                {},
                {},
                {
                    category: [
                        "\"body_weight\" is not a valid choice."
                    ]
                },
                {},
                {},
                {}
            ]
            intervention: [
                "This field is required."
            ]
        }
    ]

In [562]:
group_dict = {
            "count": 14,
            "name": "Seng2009-S_NCNSlalsöd",
            "description": "59 healthy adult Asian volunteers were included. In what follows, a non-caffeine consumer (NCCS) refers to an individual who consumed beverages\n(coffee or tea) containing <=200 mg of caffeine per day, and a regular caffeine consumer to one who consumed more >200 mg. A cup (150 mL) of coffee or tea is equivalent to approximately a caffeine dose of 70 and 25 mg respectively (9). Additionally, a non-smoker refers to an individual who did not smoke for at least 6 months prior to dosing. The study population consisted of 14 non-caffeine consumers, non-smokers (NCNS) [age (mean \u00b1 SD) 21 \u00b1 2 years, body weight 62 \u00b1 9 kg), 15 caffeine consumers, non smokers (CNS) (age 24 \u00b1 4 years, body weight 69 \u00b1 12 kg), and 30 caffeine consumer, smokers (CS) (actual consumption 10 \u00b1 4 cigarettes daily, age 23 \u00b1 5 years, body weight 71 \u00b1 19 kg).",
            "characteristic_values": [
                {
                    "count": 14,
                    "category": "sex",
                    "choice": "M"
                },
                {
                    "count": 14,
                    "category": "species",
                    "choice": "homo sapiens"
                },
                {
                    "category": "body_weight",
                    "count": 14,
                    "mean": 62.0,
                    "sd": 9.0,
                    "unit": "kg"
                },
                {
                    "count": 14,
                    "category": "ethnicity",
                    "choice": "Asian"
                },
        
            ],
            "intervention":[],
        }
client.action(document, ["groups", "create"], params=group_dict)

ErrorMessage: <Error: 400 Bad Request>
    characteristic_values: [
        {},
        {},
        {
            category: [
                "\"body_weight\" is not a valid choice."
            ]
        },
        {}
    ]

In [538]:
[type(item) for item in reference_dict.values()]

[int, str, str, str, str, str, str, list, str]

In [541]:
reference_dict

{'pmid': 12087394589,
 'sid': '012087345981',
 'name': 'Haller2002',
 'date': '2002-07-18',
 'journal': 'Clinical pharmacology and therapeutics',
 'title': 'Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.',
 'abstract': 'Botanical stimgle dose.',
 'authors': [{'first_name': 'Christine asdin', 'last_name': 'Haller'},
  {'first_name': 'Peyton', 'last_name': 'Jacob'},
  {'first_name': 'Neal L', 'last_name': 'Benowitz'}],
 'doi': ''}

In [543]:
from jsonschema import validate
reference_schema = {
     "properties":{
        "pmid": {"type" : "number"},
        "name": {"type" : "string"},
        "sid": {"type" : ["number","string"]},
        "date": {"type" : "string"},
        "journal": {"type" : "string"},
        "title": {"type" : "string"},
        "abstract": {"type" : "string"},
        "authors": {"type" : "array"},
        "doi": {"type" : "string"}
         
     }
    
}

validate(reference_dict,reference_schema)

{}

{}

In [359]:
import json
with open('/home/janekg89/Develop/Pycharm_Projects/pkdb/pkdb_app/data/Master/Studies/Kaplan1997/data.json') as f:
    data = json.load(f)


In [ ]:
client.action(document, ["references", "list"])

In [117]:
client.action(document,["references", "create"], params=data)

ParameterError: {'groups': 'This parameter is required.'}

In [62]:
charaVal = {
                    "category": "caffeine_amount",
                    "count": 8,
                    "mean": None,
                    "median": None,
                    "min": None,
                    "max": None,
                    "sd": None,
                    "se": None,
                    "cv": None,
                    "unit": None
                }
client.action(document,["characteristic_values", "create"], params=charaVal)

ErrorMessage: <Error: 400 Bad Request>
    group: [
        "This field is required."
    ]

In [35]:
reference_dict

{'pmid': 12087345,
 'sid': 'Haller2002',
 'date': '2002-07-16',
 'journal': 'Clinical pharmacology and therapeutics',
 'title': 'Pharmacology of ephedra alkaloids and caffeine after single-dose dietary supplement use.',
 'abstract': 'Botanical stimulants have disposition characteristics similar to their pharmaceutical counterparts, and they can produce significant cardiovascular responses after a single dose.',
 'authors': [{'first_name': 'Christine A', 'last_name': 'Haller'},
  {'first_name': 'Peyton', 'last_name': 'Jacob'},
  {'first_name': 'Neal L', 'last_name': 'Benowitz'}],
 'doi': '',
 'groups': [{'count': 8,
   'sid': 'Haller2002-S1',
   'description': 'Eight healthy volunteers (5 women and 3 men) from 25 to 38 years old participated in the study. Eligibility for the study was determined on the basis of medical history, brief physical examination, and screening laboratory tests that included complete blood count, serum chemistry values to assess liver and renal function, urine t

In [48]:
path ="/home/janekg89/Develop/Pycharm_Projects/pkdb/pkdb_app/data/caffeine/Subjects.tsv"

In [53]:
def subjects_load(json_reference):
    data_pd = pd.read_csv(path,delimiter='\t')
    this_data = data_pd[data_pd["study"] == json_reference["json"]["sid"]]
    json = {**json_reference["json"]}
    json["groups"] = []
    print(len(this_data))
    for name, row in this_data.iterrows():
        yield {"json": json, "group": row.to_dict()}# "reference_path": json_reference["reference_path"]}

In [54]:
data = {"json" : study_dict }
this = subjects_load(data)

In [55]:
next(this)

2


{'json': {'pmid': 16198659,
  'sid': 'Granfors2005',
  'date': '2005-11-02',
  'journal': 'Clinical pharmacology and therapeutics',
  'title': 'Oral contraceptives containing ethinyl estradiol and gestodene markedly increase plasma concentrations and effects of tizanidine by inhibiting cytochrome P450 1A2.',
  'abstract': 'OCs containing ethinyl estradiol and gestodene increase, to a clinically significant extent, the plasma concentrations and effects of tizanidine, probably mainly by inhibiting its CYP1A2-mediated presystemic metabolism. Care should be exercised when tizanidine is prescribed to OC users.',
  'groups': [],
  'authors': [{'first_name': 'Marika T', 'last_name': 'Granfors'},
   {'first_name': 'Janne T', 'last_name': 'Backman'},
   {'first_name': 'Jouko', 'last_name': 'Laitila'},
   {'first_name': 'Pertti J', 'last_name': 'Neuvonen'}],
  'doi': ''},
 'group': {'study': 'Granfors2005',
  'subjects': 'S1',
  'keywords': 'control',
  'species': 'homo sapiens',
  'strain': '-'

In [60]:
"33.7".replace('.','',1).isdigit()

True